In [1]:
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout
from tensorflow.keras.preprocessing.image import ImageDataGenerator


In [2]:

try:
    from tensorflow.keras import mixed_precision
    mixed_precision.set_global_policy('mixed_float16')
    print("Mixed precision enabled.")
except:
    print("Mixed precision not available on this setup.")

INFO:tensorflow:Mixed precision compatibility check (mixed_float16): OK
Your GPU will likely run quickly with dtype policy mixed_float16 as it has compute capability of at least 7.0. Your GPU: NVIDIA GeForce RTX 2050, compute capability 8.6
Mixed precision enabled.


In [3]:
IMG_SIZE = (224, 224)  # Keep full size
BATCH_SIZE = 8         # Much smaller batch size

In [4]:
train_datagen = ImageDataGenerator(
    rescale=1.0/255,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True
)

In [5]:
test_datagen = ImageDataGenerator(rescale=1.0/255)

In [6]:
train_generator = train_datagen.flow_from_directory(
    'Road_Anomaly_Dataset/train',
    target_size=IMG_SIZE,
    batch_size=BATCH_SIZE,
    class_mode='categorical'
)

Found 2635 images belonging to 4 classes.


In [8]:
test_generator = test_datagen.flow_from_directory(
    'Road_Anomaly_Dataset/test',
    target_size=IMG_SIZE,
    batch_size=BATCH_SIZE,
    class_mode='categorical'
)

Found 139 images belonging to 4 classes.


In [9]:
model = Sequential([
    Conv2D(32, (3,3), activation='relu', input_shape=(IMG_SIZE[0], IMG_SIZE[1], 3)),
    MaxPooling2D(pool_size=(2,2)),

    Conv2D(64, (3,3), activation='relu'),
    MaxPooling2D(pool_size=(2,2)),

    Conv2D(128, (3,3), activation='relu'),
    MaxPooling2D(pool_size=(2,2)),

    Flatten(),
    Dense(128, activation='relu'),
    Dropout(0.5),
    Dense(train_generator.num_classes, activation='softmax')
])

In [10]:
model.compile(optimizer='adam',
              loss='categorical_crossentropy',
              metrics=['accuracy'])

In [11]:
history = model.fit(
    train_generator,
    steps_per_epoch=train_generator.samples // BATCH_SIZE,
    epochs=20,
    validation_data=test_generator,
    validation_steps=test_generator.samples // BATCH_SIZE
)

Epoch 1/20
329/329 [==============================] - 128s 257ms/step - loss: 0.4457 - accuracy: 0.8386 - val_loss: 0.0024 - val_accuracy: 1.0000
Epoch 2/20
329/329 [==============================] - 29s 89ms/step - loss: 0.0662 - accuracy: 0.9775 - val_loss: 0.2005 - val_accuracy: 0.9412
Epoch 3/20
329/329 [==============================] - 26s 78ms/step - loss: 0.1030 - accuracy: 0.9699 - val_loss: 2.6748e-05 - val_accuracy: 1.0000
Epoch 4/20
329/329 [==============================] - 26s 79ms/step - loss: 0.0317 - accuracy: 0.9916 - val_loss: 9.4175e-06 - val_accuracy: 1.0000
Epoch 5/20
329/329 [==============================] - 25s 75ms/step - loss: 0.0229 - accuracy: 0.9909 - val_loss: 0.0107 - val_accuracy: 1.0000
Epoch 6/20
329/329 [==============================] - 26s 80ms/step - loss: 0.0459 - accuracy: 0.9878 - val_loss: 1.1661e-04 - val_accuracy: 1.0000
Epoch 7/20
329/329 [==============================] - 25s 77ms/step - loss: 0.0238 - accuracy: 0.9931 - val_loss: 1.3431e-

In [12]:
model.save("cnn_model_224.h5")

In [13]:
model_json = model.to_json()
with open("cnn_model_224.json",'w') as json_file:
    json_file.write(model_json)

In [14]:
from keras.models import model_from_json

In [15]:
json_file = open("cnn_model_224.json", "r")
model_json = json_file.read()
json_file.close()
model = model_from_json(model_json)
model.load_weights("cnn_model_224.h5")

In [16]:
label = ['Accident','Fight','Fire','Snatching']

In [17]:
def ef(image):
    img = load_img(image,grayscale =  True )
    feature = np.array(img)
    feature = feature.reshape(1,48,48,1)
    return feature/255.0